In [ ]:
# Install czmodel and dependencies
! pip install --upgrade pip|
! pip install "czmodel>=3.0,<3.1"

In [ ]:
# This can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Simple TF2 + Keras model for regression (to denoise the image)
This notebook shows the entire workflow of training an ANN with [TensorFlow 2](https://www.tensorflow.org/) using the Keras API and exporting the trained model to the [CZANN format](https://pypi.org/project/czmodel/) to be ready for use within the [Intellesis](https://www.zeiss.de/mikroskopie/produkte/mikroskopsoftware/zen-intellesis-image-segmentation-by-deep-learning.html) infrastructure.

* The trained model is rather simple (for demo purposes) and trained on a small test dataset.
* **Therefore, this notebook is meant to be understood as a guide for exporting trained models.**
* **The notebook does not provide instructions how to train a model correctly.**

## Imports

In [ ]:
# Required imports to train a simple TF2 + Keras model for regression and package it as CZANN.
# The CZANN can then be imported in ZEN and used for regression workflows (e.g. denoising).

# General imports
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

# Function provided by the PyPI package called czmodel (by ZEISS)
from czmodel.model_metadata import ModelMetadata, ModelSpec, ModelType
from czmodel import DefaultConverter
from czmodel.util.transforms import Shift, Scale

In [ ]:
# Optional: suppress TF warnings
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
print(tf.__version__)
print(tf.version.GIT_VERSION, tf.__version__)

## Training Pipeline
This section describes a simple training procedure that creates a trained Keras model.

* Therefore, it only represents the custom training procedure
* Such procedure will vary from case to case and will contain more sophisticated ways to generate an optimized Keras model

### Define parameters for data loading

In [ ]:
# Folder containing the input images
IMAGES_FOLDER = 'Mouse_Kidney_images/images/'

# Folder containing the ground truth regression labels
# Regression labels contain a real number for each pixel
LABELS_FOLDER = 'Mouse_Kidney_images/labels/'

# Path to the data on GitHub
GITHUB_TRAINING_DATA_PATH = 'https://raw.githubusercontent.com/zeiss-microscopy/OAD/master/Machine_Learning/notebooks/czmodel/Mouse_Kidney_images.zip' # TODO: Update this path
GITHUB_MODEL_CONVERSION_SPEC_PATH = 'https://raw.githubusercontent.com/zeiss-microscopy/OAD/master/Machine_Learning/notebooks/czmodel/regression_conversion_spec.json' # TODO Update this path

# Define the number of input color channels
# This means that the inputs are grayscale with one channel only
CHANNELS = 1 

# The size of image crops to train the model with
CROP_SIZE = 512

### Download data if it's not available on disk
If this notebook is run e.g. as a colab notebook, it does not have access to the data folder on gitub via disk access. 
In that case we need to download the data from github first.

In [ ]:
print(os.getcwd())

In [ ]:
import requests

# Download training data
if not (os.path.isdir(IMAGES_FOLDER) and os.path.isdir(LABELS_FOLDER)):
    compressed_data = './Mouse_Kidney_images.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_TRAINING_DATA_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)
        
    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')
        
# Download model conversion spec
if not os.path.isfile('regression_conversion_spec.json'):
    response = requests.get(GITHUB_MODEL_CONVERSION_SPEC_PATH, stream=True)
    with open('regression_conversion_spec.json', 'wb') as handle:
        handle.write(response.content)

### Read images
This part contains the logic to read pairs of images and label masks for training

In [ ]:
# Determine the paths of the input samples
sample_images = sorted([os.path.join(IMAGES_FOLDER, f) for f in os.listdir(IMAGES_FOLDER) 
                        if os.path.isfile(os.path.join(IMAGES_FOLDER, f))])

# Determine the paths of the corresponding labels
sample_labels = sorted([os.path.join(LABELS_FOLDER, f) for f in os.listdir(LABELS_FOLDER) 
                       if os.path.isfile(os.path.join(LABELS_FOLDER, f))])

# Load images as numpy arrays and scale to interval [0..1]
images_loaded = np.asarray(
    [
        tf.image.decode_image(
            tf.io.read_file(sample_path), channels=CHANNELS, dtype=tf.uint16
        ).numpy().astype(np.float32) / (2**16 - 1)
        for sample_path in sample_images
    ]
)

# Load the labels as numpy arrays and scale to interval [0..1]
labels_loaded = np.asarray([
    tf.image.decode_image(
            tf.io.read_file(sample_path), channels=CHANNELS, dtype=tf.uint8
        ).numpy().astype(np.float32) / (2**8 - 1)
        for sample_path in sample_labels
])

In [ ]:
# Normalize images
images_mean = images_loaded.mean(axis=(0,1,2))
images_std = images_loaded.std(axis=(0,1,2))
images_loaded = (images_loaded - images_mean) / images_std

# Normalize labels
labels_loaded = (labels_loaded - images_mean) / images_std

### Define a TensorFlow dataset to pre-process the images
Since the dataset contains very large images we need to train on smaller crops in order to not exhaust the GPU memory

In [ ]:
# Define a simple random crop transformation to train on smaller crops
def random_crop(image, mask, height, width):
    stacked = tf.concat([image, mask], axis=-1)
    stacked_cropped = tf.image.random_crop(
        stacked,
        size=tf.stack([height, width, tf.shape(image)[-1] + tf.shape(mask)[-1]], axis=0)
    )
    image_cropped = stacked_cropped[..., :tf.shape(image)[-1]]
    mask_cropped = stacked_cropped[..., tf.shape(image)[-1]:]
    return image_cropped, mask_cropped

# Define a TensorFlow dataset applying the random crop and batching the training data.
dataset = tf.data.Dataset.from_tensor_slices(
    ((images_loaded, labels_loaded))
).map(
    lambda x, y: random_crop(x, y, CROP_SIZE, CROP_SIZE)
).shuffle(10).batch(8)

### Define a simple model
This part defines a simple Keras convolutional model for regression task. It is also possible to add pre-processing layers to the model here.

In [ ]:
# Define simple Keras convolutional model
model = tf.keras.models.Sequential(
    [   # Encoder
        tf.keras.layers.Conv2D(16, 1, activation="linear", padding="same"),
        tf.keras.layers.MaxPooling2D(2, padding="same"),
        tf.keras.layers.Conv2D(16, 1, activation="linear", padding="same"),
        tf.keras.layers.MaxPooling2D(2, padding="same"),
        tf.keras.layers.Conv2D(16, 1, activation="linear", padding="same"),
        # Decoder
        tf.keras.layers.UpSampling2D(2, interpolation = 'bilinear'),
        tf.keras.layers.Conv2D(16, 1, activation="linear", padding="same"),
        tf.keras.layers.UpSampling2D(2, interpolation = 'bilinear'), 
        tf.keras.layers.Conv2D(1, 3, activation="selu", padding="same"),
    ]
)

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='mean_absolute_error')

### Fit the model to the loaded data
This part fits the model to the loaded data. In this test example we do not care about an actual evaluation of the model using validation and test datasets.

In [ ]:
def plot_history(history):
  plt.plot(history.history['loss'])
  plt.title('training loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
# Define number of training epochs
NUM_EPOCHS = 50

# Fit the model to the data
history = model.fit(dataset, epochs=NUM_EPOCHS)

# Plot the training history
plot_history(history)

In [ ]:
# Plot the training results
images_labels  = next(iter(dataset))
image = images_labels[0][0]
label = images_labels[1][0]

plt.figure(figsize=(16,16))
# Plot the inputs
plt.subplot(1,3,1)
plt.axis('off')
plt.title('Inputs')
plt.imshow(image[:,:,0], cmap='gray')

# Plot the predictions
prediction = model.predict(image)
plt.subplot(1,3,2)
plt.axis('off')
plt.title('Predictions')
plt.imshow(prediction[:,:,0,0], cmap='gray')

# Plot the ground truth labels
plt.subplot(1,3,3)
plt.axis('off')
plt.title('Ground truth labels')
plt.imshow(label[:,:,0], cmap='gray')
plt.show()

## Create a CZANN from the trained Keras model
In this section we export the trained model to the CZANN format using the czmodel library and some additional meta data all possible parameter choices are described in the [ANN model specification](https://pypi.org/project/czmodel/).

### Define Meta Data
We first define the meta data needed to run the model within the Intellesis infrastructure. The `czmodel` package offers a named tuple `ModelMetadata` that allows to either parse as JSON file as described in the [specification document](https://pypi.org/project/czmodel/) or to directly specify the parameters as shown below.

### Create a Model Specification Object
The export functions provided by the `czmodel` package expect a `ModelSpec` tuple that features the Keras model to be exported, the corresponding model meda data and optionally a license file for the model.

Therefore, we wrap our model and the `model_metadata` instance into a `ModelSpec` object.

In [ ]:
# Define dimensions - ZEN Intellesis requires fully defined spatial dimensions in the meta data of the CZANN model.
# The ZEN TilingClient uses the input shape in the meta data to infer the tile size to pass an image to the inferencer.
# Important: The tile size has to be chosen s.t. inference is possible with the minimum hardware requirements of Intellesis
# Optional: Define target spatial dimensions of the model for inference.
input_size = 1024

# Define the model metadata
model_metadata = ModelMetadata(
    input_shape=[input_size, input_size, CHANNELS],
    output_shape=[input_size, input_size, CHANNELS],
    model_type=ModelType.REGRESSION,
    model_name="Simple_Kidney_RegressionModel",
    min_overlap=[8, 8],
)

model_spec = ModelSpec(
    model=model,
    model_metadata=model_metadata,
    license_file=None
)

# Define pre-processing
preprocessing = [
    Shift(-images_mean),
    Scale(1.0 / images_std)
]

# Define post-processing
postprocessing = [
    Scale(images_std),
    Shift(images_mean)
]

### Perform model export into *.czann  file format

The converters from the `czmodel` library offers two functions to perform the actual export. 

* `convert_from_json_spec` allows to provide a JSON file containing all the information of a ModelSpec object and converts a model in SavedModel format on disk to a `.czann` file that can be loaded with ZEN.
* `convert_from_model_spec` expects a `ModelSpec` object, an output path and name and optionally target spatial dimensions for the expected input of the exported model. From this information it creates a `.czann`  file containing the specified model.

Currently, `czmodel` offers one converter for a regression model:
* DefaultConverter: Converts a model to a *.czann file.

In [ ]:
DefaultConverter().convert_from_model_spec(
    model_spec=model_spec, 
    output_path='./', 
    output_name='simple_kidney_regmodel',
    spatial_dims=(input_size, input_size),
    preprocessing=preprocessing,
    postprocessing=postprocessing
)

# In the example above there will be a ""./czmodel_output/simple_kidney_regmodel.czann" file saved on disk.

## Remarks
The generated .czann file can be directly loaded into ZEN Intellesis to perform regression tasks with the trained model.
If there is already a trained model in SavedModel format present on disk, it can also be converted by providing the path to the saved model directory instead of a Keras `Model` object. The `czmodel` library will implicitly load the model from the provided path.

The `czmodel` library also provides a `convert_from_json_spec` function that accepts a JSON file with the above mentioned meta data behind the key `ModelMetadata` which will implicitly be deserialized into a `ModelMetadata` object, the model path and optionally a license file:
```json
{
    "ModelMetadata": {
        "Type": "Regression",
        "InputShape": [1024, 1024, 1],
        "OutputShape": [1024, 1024, 1],
        "ModelName": "Regression Model From JSON",
        "MinOverlap": [8, 8]
    },
    "ModelPath": "./saved_tf2_model_output/",
    "LicenseFile": null
}
```

This information can be copied to a file e.g. in the current working directory `./model_conversion_spec.json` that also contains the trained model in SavedModel format e.g. generated by the following line:

In [ ]:
# save the trained TF2.SavedModel as a folder structure
# The folder + the JSON file can be also used to import the model in ZEN

model.save('./saved_tf2_model_output/')

In [ ]:
# This is an additional way to create a CZANN from a saved TF2 model on disk + JSON file.
# The currently recommended way to to create the CZANN directly by using czmodel.convert_from_model_spec
# the path to the TF2.SavedModel folder is defined in the JSON shown above

DefaultConverter().convert_from_json_spec(
    model_spec_path='regression_conversion_spec.json',
    output_path='./',
    output_name = 'simple_kidney_regmodel_from_json',
    spatial_dims=(input_size, input_size),
    preprocessing=preprocessing,
    postprocessing=postprocessing
)

Use the commands below from a terminal to present the notebook as a slideshow.

`
jupyter nbconvert Regression.ipynb --to slides --post serve 
    --SlidesExporter.reveal_theme=serif 
    --SlidesExporter.reveal_scroll=True 
    --SlidesExporter.reveal_transition=none
`